In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib

In [2]:
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-max-temperatures.csv'
urllib.request.urlretrieve(data_url, 'daily-max-temperatures.csv')

('daily-max-temperatures.csv', <http.client.HTTPMessage at 0x289a3249a00>)

In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [4]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
import pandas as pd

df = pd.read_csv('daily-max-temperatures.csv', sep=',',
                     infer_datetime_format=True, index_col='Date', header=0)

In [10]:
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))
SPLIT_TIME = int(len(data) * 0.8)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]
print('Total Data sebanyak : ', df.size)
print('Data Train sebesar : ', x_train.size)
print('Data Test sebesar : ', x_valid.size)

Total Data sebanyak :  3650
Data Train sebesar :  2920
Data Test sebesar :  730


In [13]:
window_size = 64
batch_size = 256
shuffle_buffer_size = 1000

train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
valid_set = windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)

In [14]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                            strides=1, padding="causal",
                            activation="relu",
                            input_shape=[None, 1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(60, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1),
        ])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
            lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
                    optimizer=optimizer,
                    metrics=["mae"])
history = model.fit(train_set, epochs=150, callbacks=[lr_schedule], validation_data=valid_set)

Epoch 1/150
12/12 [==============================] - 10s 238ms/step - loss: 0.0528 - mae: 0.2940 - val_loss: 0.0520 - val_mae: 0.2919 - lr: 1.0000e-08
Epoch 2/150
12/12 [==============================] - 1s 55ms/step - loss: 0.0528 - mae: 0.2940 - val_loss: 0.0520 - val_mae: 0.2919 - lr: 1.1220e-08
Epoch 3/150
12/12 [==============================] - 1s 54ms/step - loss: 0.0528 - mae: 0.2940 - val_loss: 0.0520 - val_mae: 0.2919 - lr: 1.2589e-08
Epoch 4/150
12/12 [==============================] - 1s 52ms/step - loss: 0.0528 - mae: 0.2940 - val_loss: 0.0520 - val_mae: 0.2919 - lr: 1.4125e-08
Epoch 5/150
12/12 [==============================] - 1s 52ms/step - loss: 0.0528 - mae: 0.2940 - val_loss: 0.0520 - val_mae: 0.2919 - lr: 1.5849e-08
Epoch 6/150
12/12 [==============================] - 1s 54ms/step - loss: 0.0528 - mae: 0.2940 - val_loss: 0.0520 - val_mae: 0.2919 - lr: 1.7783e-08
Epoch 7/150
12/12 [==============================] - 1s 54ms/step - loss: 0.0528 - mae: 0.2940 - val_los